In [3]:
# Importing modules...
import numpy as np
import cv2

import sys
sys.path.append('../..') # Go back to base directory

from modules.plot.viewer2d import Viewer2D

from modules.vision.camera import Camera
from modules.vision.epipolar_geometry import build_essential_matrix, build_fundamental_matrix

from coppeliasim_zmqremoteapi_client import RemoteAPIClient

# Init client 
client = RemoteAPIClient()    # Client object 
sim = client.getObject('sim') # Simulation object

In [4]:
n_cameras = 8
camera = [] # Camera objects list

for ID in range(n_cameras):
    # Get the vision sensor handle
    vision_sensor_handle = sim.getObject(f'/VisionSensor[{ID}]')

    # Get Coppelia's 3x4 object matrix
    object_matrix = np.array(sim.getObjectMatrix(vision_sensor_handle)).reshape((3,4))

    # X and Y axis of Coppelia's Vision Sensor are inverted
    object_matrix[:,0] *= -1 # Invert x vector column
    object_matrix[:,1] *= -1 # Invert y vector column

    camera.append(Camera(# Simulation handling
                          vision_sensor_handle=vision_sensor_handle,
   
                          # Intrinsic Parameters
                          resolution=(720,720), 
                          fov_degrees=60.0,     
          
                          # Extrinsic Parameters
                          object_matrix=object_matrix,
          
                          # Lens Distortion Model
                          distortion_model='rational',
                          distortion_coefficients=np.array([0.014, -0.003, -0.0002, -0.000003, 0.0009, 0.05, -0.007, 0.0017]),
          
                          # Image Noise Model
                          snr_dB=30 
                          ))

In [ ]:
fundamental_matrix = np.array(np.zeros((n_cameras, n_cameras, 3, 3)))

for reference in range(n_cameras):
    for auxiliary in range(n_cameras):
        if reference == auxiliary:
            continue

        E = build_essential_matrix(camera[reference].extrinsic_matrix, camera[auxiliary].extrinsic_matrix)

        F = build_fundamental_matrix(camera[reference].intrinsic_matrix, camera[auxiliary].intrinsic_matrix, E)

        fundamental_matrix[reference][auxiliary] = F

In [ ]:
# When simulation is not running, ZMQ message handling could be a bit
# slow, since the idle loop runs at 8 Hz by default.
# Setting the idle loop to run at full speed for this program
defaultIdleFps = sim.getInt32Param(sim.intparam_idle_fps)   
sim.setInt32Param(sim.intparam_idle_fps, 0)

# Simulation begins here
sim.startSimulation()

pts_images = {}

while (t := sim.getSimulationTime()) < 5:
    images = [camera[ID].get_coppelia_image(sim.getVisionSensorCharImage) for ID in range(n_cameras)]
    pts_images[t] = images    

# Restore the original idle loop frequency:
sim.setInt32Param(sim.intparam_idle_fps, defaultIdleFps)

# Simulation ends here
sim.stopSimulation()

In [ ]:
for frame, pts in enumerate(pts_images.keys()):
    if frame == 1:
        for ID in range(n_cameras):
            cv2.imshow(str(ID), pts_images[pts][ID])
            cv2.waitKey(0) 

# closing all open windows 
cv2.destroyAllWindows() 